In [1]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing.image import image_dataset_from_directory
# from keras.preprocessing.image import image_dataset_from_directory
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16

In [2]:
# pip install tf-nightly

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
# !pip install rarfile  # Install the rarfile package if it's not already installed

# import os
# from rarfile import RarFile
# from torchvision import transforms, datasets
# # Define the path to the RAR file
# rar_path = '/content/drive/MyDrive/Dataset.rar'

# # Extract the RAR file to a temporary directory
# with RarFile(rar_path, 'r') as rar:
#     rar.extractall('/tmp/dataset')

In [5]:
# applying different data augumentation methods to the training data
train_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    shear_range = 0.2,
    zoom_range = 0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip = True,
    fill_mode = 'nearest',
    validation_split = 0.2
)

valid_gen = ImageDataGenerator(
    rescale = 1/255,
    validation_split=0.2
)

In [6]:
Image_size = (255, 255)
Batch_size = 32

In [7]:
#Loading the training and validation data from the director
data_dir = "Dataset"
train_generator = train_gen.flow_from_directory(
    data_dir,
    target_size = Image_size,
    batch_size = Batch_size,
    subset= "training",
    class_mode = "categorical",
    color_mode = "rgb"
    
)

valid_generator = valid_gen.flow_from_directory(
    data_dir,
    target_size = Image_size,
    batch_size = Batch_size,
    subset= "validation",
    class_mode = "categorical",
    color_mode = "rgb"
    
    
)



Found 49197 images belonging to 39 classes.
Found 12289 images belonging to 39 classes.


In [8]:
model = VGG16(include_top=False, input_shape=(255,255,3), weights="imagenet")
for layer in model.layers:
    layer.trainable = False
x = Flatten()(model.output)
dense1 = Dense(1024, activation="relu")(x)
prediction = Dense(39, activation="softmax")(dense1)
model = Model(inputs=model.inputs, outputs=prediction)

58889256/58889256 [==============================] - 10s 0us/step


In [9]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics  = ['accuracy']
    )

In [10]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.n // train_generator.batch_size,    #The 2 slashes division return rounded integer
        epochs = 5,
        validation_data = valid_generator,
        validation_steps = valid_generator.n // valid_generator.batch_size
        )

C:\Users\aahnd\AppData\Local\Temp/ipykernel_19684/2169345027.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5
1537/1537 [==============================] - 9971s 6s/step - loss: 0.6774 - accuracy: 0.8106 - val_loss: 0.4007 - val_accuracy: 0.8667
Epoch 2/5
1537/1537 [==============================] - 9787s 6s/step - loss: 0.3601 - accuracy: 0.8807 - val_loss: 0.3325 - val_accuracy: 0.8942
Epoch 3/5
1537/1537 [==============================] - 9522s 6s/step - loss: 0.3123 - accuracy: 0.8971 - val_loss: 0.3092 - val_accuracy: 0.9011
Epoch 4/5
1537/1537 [==============================] - 9443s 6s/step - loss: 0.2728 - accuracy: 0.9104 - val_loss: 0.3119 - val_accuracy: 0.9056
Epoch 5/5
1537/1537 [==============================] - 9719s 6s/step - loss: 0.2643 - accuracy: 0.9146 - val_loss: 0.2524 - val_accuracy: 0.9282


In [11]:
model.save('final1_model.h5')